In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np 
import os 
import matplotlib.pyplot as plt

In [3]:
# Sample connection
from pyspark.sql import SparkSession

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("Cleaning connection")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "2g")
    .config("spark.executor.memory", "4g")
    .getOrCreate()
)

22/08/16 18:41:09 WARN Utils: Your hostname, DESKTOP-L9KIK4G resolves to a loopback address: 127.0.1.1; using 172.19.228.94 instead (on interface eth0)
22/08/16 18:41:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/16 18:41:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [178]:
# Looking at the weather data and keeping columns with data
df = pd.read_csv('../data/laguardia.csv')
df.rename(str.lower, axis='columns', inplace=True)
df = df[df.report_type.isin(['FM-12', 'FM-15', 'FM-16'])]
df.drop('report_type', axis=1, inplace=True)
df.head()

/tmp/ipykernel_17460/3092311829.py:2: DtypeWarning: Columns (44,45,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/laguardia.csv')


,station,date,source,latitude,longitude,elevation,name,call_sign,quality_control,wnd,...,oc1,od1,oe1,oe2,oe3,rh1,rh2,rh3,rem,eqd
0,72503014732,2019-01-01T00:00:00,4,40.77944,-73.88035,3.4,"LAGUARDIA AIRPORT, NY US",99999,V020,"130,1,N,0041,1",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SYN09872503 11358 81308 10067 20050 30189 4020...,NaN
1,72503014732,2019-01-01T00:49:00,6,40.77944,-73.88035,3.4,"LAGUARDIA AIRPORT, NY US",KLGA,V030,"140,5,N,0067,5",...,"0103,5",NaN,NaN,NaN,NaN,NaN,NaN,NaN,MET13312/31/18 19:49:03 SPECI KLGA 010049Z 140...,NaN
2,72503014732,2019-01-01T00:51:00,7,40.77944,-73.88035,3.4,"LAGUARDIA AIRPORT, NY US",KLGA,V030,"130,5,N,0067,5",...,"0103,5",NaN,NaN,NaN,NaN,NaN,NaN,NaN,MET13912/31/18 19:51:01 METAR KLGA 010051Z 130...,NaN
3,72503014732,2019-01-01T01:15:00,7,40.77944,-73.88035,3.4,"LAGUARDIA AIRPORT, NY US",KLGA,V030,"140,5,N,0046,5",...,"0088,5",NaN,NaN,NaN,NaN,NaN,NaN,NaN,MET11212/31/18 20:15:01 SPECI KLGA 010115Z 140...,NaN
4,72503014732,2019-01-01T01:25:00,7,40.77944,-73.88035,3.4,"LAGUARDIA AIRPORT, NY US",KLGA,V030,"130,5,N,0051,5",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MET11912/31/18 20:25:01 SPECI KLGA 010125Z 130...,NaN


In [179]:
# keeping potentially relevant columns
df = df[['date', 'wnd', 'vis', 'tmp', 'dew', 'slp', 'ga1',]]

# loosely transcribed meaning of columns:

# wnd = wind angle clockwise,  quality code, wind obs type, WIND-OBSERVATION speed rate (x10), quality # 9999 missing

# vis = visibility distance, quality, variability, obs qual  # 999999=missing

# tmp = temp (x10), #+9999=missing
# dew = dewpoint(x10), #+9999=missing [dew point temperature The temperature to which a given parcel of air must be cooled at constant pressure and water vapor content in order for saturation to occur

# slp = atm pressure (x10), # 99999 = missing           [seal level pressure]
# ga1 = cloud cover (scale 0-8, 9&10=obstructed), quality, base height, quality, cloud type, quality


# Maybe interesting columns:
# mw1 = weather name, quality
# oc1 = kui suured tuulepuhangud on (x10), quality

# df.drop(['source', 'station', 'latitude', 'longitude', 'name', 'elevation', 'call_sign', 'quality_control', 'cig', 'aa1', 'aa2', 'aa3', 'ab1', 'ad1', 'ae1', 'ah1', 'ah2', 'ah3', 'ah4', 'ah5', 'ah6', 'ai1', 'ai2', 'ai3', 'ai4', 'ai5', 'ai6', 'aj1', 'ak1', 'al1', 'am1', 'an1', 'at1', 'at2', 'at3', 'at4', 'at5', 'at6', 'at7', 'at8', 'au1', 'au2', 'au3', 'au4', 'aw1', 'aw2', 'aw3', 'aw4', 'aw5','ax1', 'ax2', 'ax3', 'ax4', 'ed1', 'ga2', 'ga3', 'gd1', 'gd2', 'gd3', 'gd4', 'ge1', 'gf1', 'ka1', 'ka2', 'kb1','kb2','kb3','kc1','kc2','kd1','kd2','ke1','kg1','kg2','ma1','md1', 'mf1','mg1','mh1','mk1','mv1','mw1','mw2','mw3','oc1','od1','oe1','oe2','oe3','rh1','rh2','rh3','rem','eqd'], axis=1)
df

,date,wnd,vis,tmp,dew,slp,ga1
0,2019-01-01T00:00:00,"130,1,N,0041,1","008000,1,9,9","+0067,1","+0050,1","10201,1","99,9,+00250,1,99,9"
1,2019-01-01T00:49:00,"140,5,N,0067,5","003219,5,N,5","+0070,5","+0050,5","99999,9","07,5,+00244,5,99,9"
2,2019-01-01T00:51:00,"130,5,N,0067,5","003219,5,N,5","+0067,5","+0050,5","10181,5","07,5,+00244,5,99,9"
3,2019-01-01T01:15:00,"140,5,N,0046,5","011265,5,N,5","+0072,5","+0056,5","99999,9","07,5,+00244,5,99,9"
4,2019-01-01T01:25:00,"130,5,N,0051,5","002414,5,N,5","+0072,5","+0056,5","99999,9","07,5,+00244,5,99,9"
...,...,...,...,...,...,...,...
14076,2019-12-31T20:51:00,"240,5,N,0046,5","016093,5,N,5","+0072,5","+0011,5","10028,5","04,5,+01036,5,99,9"
14077,2019-12-31T21:00:00,"240,1,N,0046,1","016000,1,9,9","+0072,1","+0011,1","10028,1","99,9,+01250,1,99,9"
14078,2019-12-31T21:51:00,"250,5,N,0062,5","016093,5,N,5","+0072,5","+0000,5","10033,5","02,5,+00914,5,99,9"
14079,2019-12-31T22:51:00,"240,5,N,0046,5","016093,5,N,5","+0072,5","+0006,5","10036,5","07,5,+00914,5,99,9"


In [180]:
# spliting out the relevant information
df['date'] = df['date'] = pd.to_datetime(df['date'])
df['wnd'] = df['wnd'].str.split(',', expand=True)[3].astype(np.uint32)      # unit: m/s, scaling factor:10,     missing: 9999
df['vis'] = df['vis'].str.split(',', expand=True)[0].astype(np.uint32)      # unit: m,                          
df['tmp'] = df['tmp'].str.split(',', expand=True)[0].astype(np.int32)       # unit: C, scaling factor:10,       missing: 9999
df['dew'] = df['dew'].str.split(',', expand=True)[0].astype(np.int32)       # unit: C, scaling factor:10,       missing: 9999
df['atm'] = df['slp'].str.split(',', expand=True)[0].astype(np.uint32)      # unit: hP, scaling factor:10,      missing: 99999
df['ga1'].fillna('99,x', inplace=True)
df['cc'] = df['ga1'].str.split(',', expand=True)[0].astype(np.uint32).astype('category') # cloud coverage: unit: okta (0 clear -> 8 covered), 99 missing
df.drop(['slp', 'ga1'], axis=1, inplace=True)

In [184]:
df[df.isnull().any(axis=1)] # no NaN left

,date,wnd,vis,tmp,dew,atm,cc


In [187]:
print(df.dtypes)
df.describe()

date    datetime64[ns]
wnd             uint32
vis             uint32
tmp              int32
dew              int32
atm             uint32
cc            category
dtype: object


,wnd,vis,tmp,dew,atm
count,13691.000000,13691.000000,13691.000000,13691.000000,13691.000000
mean,49.218538,14097.872982,134.312979,62.898693,23881.401505
std,88.596347,9506.640887,176.095325,247.540883,32309.299094
min,0.000000,0.000000,-156.000000,-272.000000,9858.000000
25%,31.000000,16000.000000,50.000000,-17.000000,10125.000000
50%,46.000000,16093.000000,133.000000,72.000000,10186.000000
75%,62.000000,16093.000000,217.000000,144.000000,10264.000000
max,9999.000000,999999.000000,9999.000000,9999.000000,99999.000000


In [202]:
print(df.shape)
df = df[(df['wnd'] != 9999)]
df = df[(df['vis'] != 999999)]
df = df[(df['tmp'] != 9999)] 
df = df[(df['dew'] != 9999)] 
df = df[(df['atm'] != 99999)]
print(df.shape)

(11598, 7)
(11598, 7)


In [203]:
df.describe()

,wnd,vis,tmp,dew,atm
count,11598.000000,11598.000000,11598.000000,11598.000000,11598.000000
mean,48.211847,14861.065097,133.691499,50.988533,10168.289446
std,24.796421,3461.115500,100.612244,106.230097,81.400765
min,0.000000,0.000000,-156.000000,-272.000000,9858.000000
25%,31.000000,16000.000000,50.000000,-33.000000,10114.000000
50%,46.000000,16093.000000,133.000000,61.000000,10166.000000
75%,62.000000,16093.000000,222.000000,139.000000,10225.000000
max,175.000000,16093.000000,372.000000,239.000000,10408.000000


In [220]:
day_count = 0

for group,df_group in df.groupby(df['date'].dt.date):
    day_count += 1
    h_count = 0
    for h, h_group in df_group.groupby(df_group['date'].dt.hour):
        h_count += 1
    if h_count != 24:
        print(f"Problem on date {group}, missing {24-h_count}")
if day_count != (df['date'].iloc[-1] - df['date'].iloc[0]).days:
    print("Some days are missing from dataset")


Problem on date 2019-02-08, missing 1
Problem on date 2019-03-01, missing 1
Problem on date 2019-03-04, missing 10
Problem on date 2019-03-13, missing 1
Problem on date 2019-03-16, missing 2
Problem on date 2019-05-10, missing 1
Problem on date 2019-05-31, missing 1
Problem on date 2019-09-20, missing 5
Problem on date 2019-09-21, missing 16
Problem on date 2019-09-27, missing 2
Problem on date 2019-10-02, missing 1
Problem on date 2019-10-25, missing 1
Problem on date 2019-11-22, missing 1
Some days are missing from dataset


In [209]:
df

,date,wnd,vis,tmp,dew,atm,cc
0,2019-01-01 00:00:00,41,8000,67,50,10201,99
2,2019-01-01 00:51:00,67,3219,67,50,10181,7
6,2019-01-01 01:51:00,46,4828,72,61,10164,7
9,2019-01-01 02:51:00,62,8047,78,61,10139,4
10,2019-01-01 03:00:00,62,8000,78,61,10139,99
...,...,...,...,...,...,...,...
14076,2019-12-31 20:51:00,46,16093,72,11,10028,4
14077,2019-12-31 21:00:00,46,16000,72,11,10028,99
14078,2019-12-31 21:51:00,62,16093,72,0,10033,2
14079,2019-12-31 22:51:00,46,16093,72,6,10036,7


In [ ]:
# TODO something with missing values
# usually 99999

In [45]:
df['rh3'].value_counts()
#df[df.al1.notna()].drop(['source', 'station', 'latitude', 'longitude', 'name', 'elevation', 'call_sign', 'quality_control', 'cig', 'aa1', 'aa2', 'aa3', 'ab1', 'ad1', 'ae1', 'ah1','ah2', 'ah3','ah4','ah5','ah6', 'ai1','ai2', 'ai3','ai4','ai5','ai6'] , axis=1)


Series([], Name: rh3, dtype: int64)